## Bike Sharing Assignment

<b>Problem Statement</b><br/>
A US bike-sharing provider BoomBikes has recently suffered considerable dips in their revenues due to the ongoing Corona pandemic. The company is finding it very difficult to sustain in the current market scenario. So, it has decided to come up with a mindful business plan to be able to accelerate its revenue as soon as the ongoing lockdown comes to an end, and the economy restores to a healthy state.

In such an attempt, BoomBikes aspires to understand the demand for shared bikes among the people after this ongoing quarantine situation ends across the nation due to Covid-19. They have planned this to prepare themselves to cater to the people's needs once the situation gets better all around and stand out from other service providers and make huge profits.

They have contracted a consulting company to understand the factors on which the demand for these shared bikes depends. Specifically, they want to understand the factors affecting the demand for these shared bikes in the American market. The company wants to know:
- Which variables are significant in predicting the demand for shared bikes.
- How well those variables describe the bike demands 

### Step1: Reading and Understanding the data

In [1]:
# Importing all the necessary Libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Loading the dataset

bikes_df = pd.read_csv('day.csv')
bikes_df.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,01-01-2018,1,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985
1,2,02-01-2018,1,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801
2,3,03-01-2018,1,0,1,0,1,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349
3,4,04-01-2018,1,0,1,0,2,1,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562
4,5,05-01-2018,1,0,1,0,3,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600


In [3]:
# Checking the shape of the dataframe
bikes_df.shape

(730, 16)

#### No. of rows = 730
#### No. of columns = 16

In [4]:
# Checking the datatypes of dataframe

bikes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     730 non-null    int64  
 1   dteday      730 non-null    object 
 2   season      730 non-null    int64  
 3   yr          730 non-null    int64  
 4   mnth        730 non-null    int64  
 5   holiday     730 non-null    int64  
 6   weekday     730 non-null    int64  
 7   workingday  730 non-null    int64  
 8   weathersit  730 non-null    int64  
 9   temp        730 non-null    float64
 10  atemp       730 non-null    float64
 11  hum         730 non-null    float64
 12  windspeed   730 non-null    float64
 13  casual      730 non-null    int64  
 14  registered  730 non-null    int64  
 15  cnt         730 non-null    int64  
dtypes: float64(4), int64(11), object(1)
memory usage: 91.4+ KB


#### No missing values observed in all the columns of the dataframe

### Step 2: Data Cleansing

In [5]:
# Dropping columns like instant, dteday, casual, registered
# instant is an index
# Ignoring dteday as we already have year & month as separate columns
# Ignoring casual & registered as cnt already has total count considering both casual & registered users

bikes_df.drop(['instant', 'dteday', 'casual', 'registered'], axis=1, inplace=True)

In [6]:
# Dropping columns holiday as 'working_day' as information about holiday & weekend

bikes_df.drop(['holiday'], axis=1, inplace=True)

In [7]:
bikes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   season      730 non-null    int64  
 1   yr          730 non-null    int64  
 2   mnth        730 non-null    int64  
 3   weekday     730 non-null    int64  
 4   workingday  730 non-null    int64  
 5   weathersit  730 non-null    int64  
 6   temp        730 non-null    float64
 7   atemp       730 non-null    float64
 8   hum         730 non-null    float64
 9   windspeed   730 non-null    float64
 10  cnt         730 non-null    int64  
dtypes: float64(4), int64(7)
memory usage: 62.9 KB


In [8]:
# Renaming the columns for better readability

bikes_df.rename(columns={'yr': 'year', 'mnth': 'month', 'hum': 'humidity', 'cnt': 'count', 'weathersit': 'weather'}, inplace=True)

In [9]:
bikes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   season      730 non-null    int64  
 1   year        730 non-null    int64  
 2   month       730 non-null    int64  
 3   weekday     730 non-null    int64  
 4   workingday  730 non-null    int64  
 5   weather     730 non-null    int64  
 6   temp        730 non-null    float64
 7   atemp       730 non-null    float64
 8   humidity    730 non-null    float64
 9   windspeed   730 non-null    float64
 10  count       730 non-null    int64  
dtypes: float64(4), int64(7)
memory usage: 62.9 KB


In [ ]:
# Mapping the actual values for categorical variables for visualization and splitting to dummy variables

bikes_df.season = bikes_df.season.map({1:'spring', 2:'summer', 3:'fall', 4:'winter'})
bikes_df.month = bikes_df.month.map({1:'jan', 2:'feb', 3:'mar', 4:'apr', 5:'may', 6:'jun', 7:'jul', 8:'aug',9:'sep', 10:'oct', 11:'nov', 12:'dec',})
bikes_df.weekday = bikes_df.weekday.map({0:'sunday', 1:'monday', 2:'tuesday', 3:'wednesday', 4:'thursday', 5:'friday', 6:'saturday'})
bikes_df.weather = bikes_df.weather.map({1:'clear', 2:'mist', 3:'light_snow', 4:'heavy_rain'})

In [ ]:
bikes_df.head()

In [ ]:
# Checking for unique values in each columns

bikes_df.nunique(axis=0)

### Step 3: Data Visualization

In [ ]:
bikes_df.info()

In [ ]:
# Visualising numerical variables using pairplot

num_cols = ['temp', 'atemp', 'humidity', 'windspeed', 'count']

sns.pairplot(bikes_df[num_cols])
plt.show()

- Count seems to have more correlation with temp and atemp variables

In [ ]:
# Plotting the correlation using heatmap

sns.heatmap(bikes_df[num_cols].corr(), annot=True)
plt.show()

In [ ]:
# we can observe that temp and atemp variables have high correlation. So, dropping atemp column

bikes_df.drop('atemp', axis=1, inplace=True)

In [ ]:
bikes_df.info()

In [ ]:
# Visualizing categorical variables. Plotting box plots for those.

plt.figure(figsize=(20, 12))
plt.subplot(2,3,1)
sns.boxplot(x = 'season', y = 'count', data = bikes_df)
plt.subplot(2,3,2)
sns.boxplot(x = 'year', y = 'count', data = bikes_df)
plt.subplot(2,3,3)
sns.boxplot(x = 'month', y = 'count', data = bikes_df)
plt.subplot(2,3,4)
sns.boxplot(x = 'weekday', y = 'count', data = bikes_df)
plt.subplot(2,3,5)
sns.boxplot(x = 'workingday', y = 'count', data = bikes_df)
plt.subplot(2,3,6)
sns.boxplot(x = 'weather', y = 'count', data = bikes_df)
plt.show()

#### Insights

- In fall season, the rentals are higher
- The rentals have increased from 2018 to 2019
- Months: Jul to Sep, the rentals are higher
- When weather is clear, the rentals are higher
- Don't see any significant trend from weekday and workingday bar plots

### Step 4: Data Preprocessing

#### Creating Dummy Variables

In [ ]:
# Creating dummy variables for category variables for season, month, weekday, weathersituation

seasons = pd.get_dummies(bikes_df.season, drop_first=True)
months = pd.get_dummies(bikes_df.month, drop_first=True)
days = pd.get_dummies(bikes_df.weekday, drop_first=True)
weather = pd.get_dummies(bikes_df.weather, drop_first=True)

# Adding results to the original dataframe

bikes_df = pd.concat([bikes_df, seasons], axis=1)
bikes_df = pd.concat([bikes_df, months], axis=1)
bikes_df = pd.concat([bikes_df, days], axis=1)
bikes_df = pd.concat([bikes_df, weather], axis=1)


# Dropping original columns as we have dummy columns
bikes_df.drop(['season', 'month', 'weekday', 'weather'], axis=1, inplace=True)


bikes_df.head()

### Step 5: Splitting data into Train and Test sets

In [ ]:
# Splitting the data into train and test sets in the ratio 70:30

# We specify this so that the train and test data set always have the same rows, respectively
np.random.seed(0)
df_train, df_test = train_test_split(bikes_df, train_size=0.7, test_size=0.3, random_state=100)

print(df_train.shape)
print(df_test.shape)

#### Rescaling feature using Min-Max scaling

In [ ]:
scaler = MinMaxScaler()

In [ ]:
# Apply scaler() to all columns except the 'yes-no' and 'dummy' variables

num_cols = ['temp', 'humidity', 'windspeed', 'count']
df_train[num_cols] = scaler.fit_transform(df_train[num_cols])

df_train.head()

In [ ]:
df_train.columns

In [ ]:
df_train.describe()

#### Dividing into X and Y sets for model building

In [ ]:
y_train = df_train.pop('count')
X_train = df_train

In [ ]:
print(X_train.shape)
print(y_train.shape)

### Step 6: LinearRegression with Recursive Feature Elimination

In [ ]:
# Running RFE with the output number of the variables equal to 10

lm = LinearRegression()
lm.fit(X_train, y_train)

# Run RFE
rfe = RFE(lm, 10)
rfe = rfe.fit(X_train, y_train)

In [ ]:
list(zip(X_train.columns, rfe.support_, rfe.ranking_))

In [ ]:
cols = X_train.columns[rfe.support_]
cols

#### Insights from RFE:
- Temp, Windspeed, Humidity, Seasons(Spring), Year, Months(Jul, Sep), WeatherSit(Light_snow, Mist), Weekday(Saturday) are considered

In [ ]:
# Columns which are omitted by RFE
X_train.columns[~rfe.support_]

#### Building model using statsmodel, for detailed statistics

In [ ]:
X_train_rfe = X_train[cols]

# Adding a constant variable for statsmodel
X_train_rfe = sm.add_constant(X_train_rfe)

# Running the linear regression model (OLS)
lr_sm = sm.OLS(y_train, X_train_rfe).fit()

lr_sm.params

In [ ]:
print(lr_sm.summary())

#### Insights:
- R-squared = 0.829
- Adj. R-squared = 0.825
- p-value of saturday is on a higher side 0.066. So, let check VIF too.

In [ ]:
# Dropping the constant variable from the dataframe 

X_train_rfe = X_train_rfe.drop('const', axis=1)

#### Checking VIF

In [ ]:
# Check for the VIF values of the feature variables. 

vif = pd.DataFrame()
vif['Features'] = X_train_rfe.columns # Ignoring the const 
vif['VIF'] = [variance_inflation_factor(X_train_rfe.values, i) for i in range(X_train_rfe.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by='VIF', ascending=False)
vif

In [ ]:
# Dropping variable 'saturday' because of higher p-value of 0.066

X_train_sm = X_train_rfe.drop('saturday',axis=1)

In [ ]:
# bulding another linear model with the new dataset

X_train_sm = sm.add_constant(X_train_sm)

lr2_sm = sm.OLS(y_train, X_train_sm).fit()

In [ ]:
print(lr2_sm.summary())

#### Insights

- Adj.R-squared value is same as previous model 0.825 after dropping 'saturday' variable
- p-value are low indicating all variables are significant
- Noticed some of the coefficients have negative sign

In [ ]:
# Performing VIF

X_train_sm = X_train_sm.drop('const', axis=1)

vif = pd.DataFrame()
vif['Features'] = X_train_sm.columns # Ignoring the const 
vif['VIF'] = [variance_inflation_factor(X_train_sm.values, i) for i in range(X_train_sm.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by='VIF', ascending=False)
vif

#### Insights
- We generally want a VIF that is less than 5. So there are clearly some variables we need to drop. 
- So, we will drop 'humidity' and re-run the model

In [ ]:
# Dropping variable 'humidity' because of high VIF of 14.88
X_train_sm = X_train_sm.drop('humidity',axis=1)

# bulding another linear model with the newer dataset
X_train_sm = sm.add_constant(X_train_sm)
lr3_sm = sm.OLS(y_train, X_train_sm).fit()

# summary statistics of newer model
print(lr3_sm.summary())

In [ ]:
# Performing VIF

X_train_sm = X_train_sm.drop('const', axis=1)

vif = pd.DataFrame()
vif['Features'] = X_train_sm.columns # Ignoring the const 
vif['VIF'] = [variance_inflation_factor(X_train_sm.values, i) for i in range(X_train_sm.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by='VIF', ascending=False)
vif

#### Insights:
- For the lr3_sm model, all the p-values are low which indicates all variables are significant
- Also, VIF values are less than 5.
- Adj Rsquare value is 0.821

</br>
VIFs and p-values both are within an acceptable range. So, using lr3_sm model to make predictions

### Step 7: Residual Analysis of the train data

In [ ]:
X_train_sm = sm.add_constant(X_train_sm)

# predicting using the latest model lr3_sm model
y_train_count = lr3_sm.predict(X_train_sm)

# residuals
res = y_train - y_train_count

In [ ]:
# Plot the histogram of the error terms
fig = plt.figure()
sns.distplot(res, bins = 20)
fig.suptitle('Error Terms', fontsize = 20)# Plot heading 
plt.xlabel('Errors', fontsize = 18)       # X-Label

#### Insights:
- The error terms of the latest model (lr3_sm) gives a normal distribution with mean around 0.

In [ ]:
# Looking for patterns in the residuals

plt.scatter(np.arange(0,len(X_train_sm),1), res)
plt.show()

- We are confident that the model fit isn't by chance, and has decent predictive power. 
- Although, the variance of residuals increasing with X indicates that there is significant variation that this model is unable to explain.
- As you can see, the regression line is a pretty good fit to the data

### Step 8: Making predictions using final model

In [ ]:
# Applying scaling on the test set

num_cols = ['temp', 'humidity', 'windspeed', 'count']
df_test[num_cols] = scaler.fit_transform(df_test[num_cols])

df_test.head()

#### Dividing into X_test and y_test 

In [ ]:
y_test = df_test.pop('count')
X_test = df_test[cols]

print(y_test.head())
print(X_test.head())

In [ ]:
# Adding constant variable to test dataframe
X_test_sm = sm.add_constant(X_test)

In [ ]:
# Dropping variables which were dropped during model training from X_test 

X_test_sm = X_test_sm.drop(['humidity', 'saturday'],axis=1)

In [ ]:
y_pred_sm = lr3_sm.predict(X_test_sm)
y_pred_sm

### Step 9: Model Evalation

In [ ]:
# Plotting y_test and y_pred to understand the spread

fig = plt.figure()
plt.scatter(y_test, y_pred_sm)
fig.suptitle('y_test vs y_pred', fontsize = 20)              # Plot heading 
plt.xlabel('y_test', fontsize = 18)                          # X-label
plt.ylabel('y_pred', fontsize = 16) 

#### Insights
- The y_test vs y_pred spread shows the linearity

In [ ]:
print(lr3_sm.summary())

### The equation of the best fit line is:

$ count = 0.304 + 0.236 * year + 0.394 * temp - 0.153 * windspeed - 0.146 * spring - 0.073 * jul + 0.053 * sep - 0.275 * lightsnow - 0.080 * mist $

In [ ]:
# rsquared score on test set
print(r2_score(y_test, y_pred_sm))

#### Rsquared on train & test sets with the final model

- Rsquared score on train set: 0.824
- Rsquared score on test set: 0.789

</br>
Model does a decent job on test set

Adj. Rsquared

$ Adj. R2 = 1−(1−R2)∗(n−1)/(n−p−1) $

In [ ]:
X_test_sm.shape

In [ ]:
n = X_test_sm.shape[0]
p = X_test_sm.shape[1]

adj_r2 = 1 - (1-r2_score(y_test, y_pred_sm))* (n-1)/(n-p-1)
adj_r2

### Conclusions

The following are the variables which are significant in predicting the demand for shared bikes.
  - Year
  - Temp
  - Windspeed
  - Spring season
  - Jul & Sep months
  - Light snow/rain & Mist weather

How well those variables describe the bike demands
-  $ count = 0.304 + 0.236 * year + 0.394 * temp - 0.153 * windspeed - 0.146 * spring - 0.073 * jul + 0.053 * sep - 0.275 * lightsnow - 0.080 * mist $